# Exercício 4

Seis trabalhadores devem ser designados para seis diferentes trabalhos, cada qual devendo ser executado em um tipo diferente de máquina. Registros passados fornecem as performances individuais para os seis trabalhadores, em minutos conforme o quadro apresentado a seguir. O objetivo é designar os indivíduos aos trabalhos de tal maneira que o tempo seja minimizado.

|    **i\j**   	| **Tarefa 1** 	| **Tarefa 2** 	| **Tarefa 3** 	| **Tarefa 4** 	| **Tarefa 5** 	| **Tarefa 6** 	|
|:------------:	|:------------:	|:------------:	|:------------:	|:------------:	|:------------:	|:------------:	|
|  **Arinei**  	|      13      	|      22      	|      19      	|      21      	|      16      	|      20      	|
|   **Deisi**  	|      18      	|      17      	|      24      	|      18      	|      22      	|      27      	|
|   **Luzia**  	|      20      	|      22      	|      23      	|      24      	|      17      	|      31      	|
|   **Neiva**  	|      14      	|      19      	|      13      	|      30      	|      23      	|      22      	|
|   **Paulo**  	|      21      	|      14      	|      17      	|      25      	|      15      	|      23      	|
| **Wladimir** 	|      17      	|      23      	|      18      	|      20      	|      16      	|      24      	|

# Solução

## Modelo Matemático

### Função Objetivo

$$ min \ Z = \sum_{i \in I} \sum_{j \in J} c_{ij}x_{ij} $$

### Sujeito a

$$
\sum_{i \in I} x_{ij} = 1, \ \forall j \in J \ \dots \ (1)
$$

$$
\sum_{j \in J} x_{ij} = 1, \ \forall i \in I \ \dots \ (2)
$$

$$
x_{ij} \in \set{0, 1}, \forall i \in I, \forall j \in J \ \dots \ (3)
$$

In [1]:
# --- Imports --- #
import pyomo.environ as pyo
import pandas as pd
import numpy as np

In [2]:
# --- Declaração dos dados de entrada --- #
# Matriz de custos
matriz_custos = np.array([[13, 22, 19, 21, 16, 20],
                          [18, 17, 24, 18, 22, 27],
                          [20, 22, 23, 24, 17, 31],
                          [14, 19, 13, 30, 23, 22],
                          [21, 14, 17, 25, 15, 23],
                          [17, 23, 18, 20, 16, 24]])
lista_tarefas = ['Tarefa 1', 'Tarefa 2', 'Tarefa 3', 'Tarefa 4', 'Tarefa 5', 'Tarefa 6'] # Lista de tarefas
lista_pessoas = ['Arinei', 'Deisi', 'Luzia', 'Neiva', 'Paulo', 'Wladimir'] # Lista de pessoas
custos_designacao = {(lista_pessoas[j], lista_tarefas[i]): matriz_custos[j, i] for i in range(len(lista_tarefas)) for j in range(len(lista_pessoas))} # Dicionário de custos


In [3]:
# --- Declaração do Modelo Matemático --- #
modelo = pyo.ConcreteModel()

In [4]:
# --- Declaração dos conjuntos de Iteração --- #
modelo.tarefas = pyo.Set(initialize=lista_tarefas) # Conjunto de tarefas
modelo.pessoas = pyo.Set(initialize=lista_pessoas) # Conjunto de pessoas

In [5]:
# --- Declaração das Variáveis de Decisão --- #
modelo.x = pyo.Var(modelo.pessoas, modelo.tarefas, within=pyo.Binary) # x_{ij} = 1 se a pessoa i é designada para a tarefa j, 0 caso contrário.

In [6]:
# --- Declaração dos Parâmetros do Modelo --- #
modelo.custos = pyo.Param(modelo.pessoas, modelo.tarefas, initialize=custos_designacao) # Custo de designação da pessoa i para a tarefa j

In [8]:
# --- Declaração da Função Objetivo --- #
def funcao_objetivo(modelo):
    return sum(modelo.custos[i, j] * modelo.x[i, j] for i in modelo.pessoas for j in modelo.tarefas)
modelo.objetivo = pyo.Objective(rule=funcao_objetivo, sense=pyo.minimize)

In [11]:
# --- Declaração das Restrições do Modelo --- #
def restricao_1(modelo, j):
    return sum(modelo.x[:, j]) == 1
# Cada tarefa j deve ser designada a exatamente uma pessoa i.
modelo.restricao_1 = pyo.Constraint(modelo.tarefas, rule=restricao_1)
# ---
def restricao_2(modelo, i):
    return sum(modelo.x[i, :]) == 1
# Cada pessoa i deve ser designada a exatamente uma tarefa j.
modelo.restricao_2 = pyo.Constraint(modelo.pessoas, rule=restricao_2)

In [13]:
# --- Declaração do Solver --- #
solver = pyo.SolverFactory('gurobi')
solver.solve(modelo)

{'Problem': [{'Name': 'x1', 'Lower bound': 99.0, 'Upper bound': 99.0, 'Number of objectives': 1, 'Number of constraints': 12, 'Number of variables': 36, 'Number of binary variables': 36, 'Number of integer variables': 36, 'Number of continuous variables': 0, 'Number of nonzeros': 72, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.013000011444091797', 'Error rc': 0, 'Time': 1.426877737045288}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [14]:
# --- Extração dos Resultados --- #
dados_designacao = [{'Pessoa': i, 'Tarefa': j, 'Designado': val}
                    for (i, j), val in modelo.x.extract_values().items()]
# ---
resultados = pd.DataFrame(dados_designacao).pivot(index='Pessoa', columns='Tarefa', values='Designado')

In [16]:
resultados.to_excel('_ex_04_resultados_modelo_matematico.xlsx')

In [17]:
print(f'Função Objetivo: {modelo.objetivo()}') # Valor da função objetivo

Função Objetivo: 99.0


In [18]:
dados_designacao

[{'Pessoa': 'Arinei', 'Tarefa': 'Tarefa 1', 'Designado': 0.0},
 {'Pessoa': 'Arinei', 'Tarefa': 'Tarefa 2', 'Designado': -0.0},
 {'Pessoa': 'Arinei', 'Tarefa': 'Tarefa 3', 'Designado': -0.0},
 {'Pessoa': 'Arinei', 'Tarefa': 'Tarefa 4', 'Designado': -0.0},
 {'Pessoa': 'Arinei', 'Tarefa': 'Tarefa 5', 'Designado': -0.0},
 {'Pessoa': 'Arinei', 'Tarefa': 'Tarefa 6', 'Designado': 1.0},
 {'Pessoa': 'Deisi', 'Tarefa': 'Tarefa 1', 'Designado': -0.0},
 {'Pessoa': 'Deisi', 'Tarefa': 'Tarefa 2', 'Designado': -0.0},
 {'Pessoa': 'Deisi', 'Tarefa': 'Tarefa 3', 'Designado': -0.0},
 {'Pessoa': 'Deisi', 'Tarefa': 'Tarefa 4', 'Designado': 1.0},
 {'Pessoa': 'Deisi', 'Tarefa': 'Tarefa 5', 'Designado': -0.0},
 {'Pessoa': 'Deisi', 'Tarefa': 'Tarefa 6', 'Designado': -0.0},
 {'Pessoa': 'Luzia', 'Tarefa': 'Tarefa 1', 'Designado': -0.0},
 {'Pessoa': 'Luzia', 'Tarefa': 'Tarefa 2', 'Designado': -0.0},
 {'Pessoa': 'Luzia', 'Tarefa': 'Tarefa 3', 'Designado': -0.0},
 {'Pessoa': 'Luzia', 'Tarefa': 'Tarefa 4', 'Designad